In [2]:
from Bio import SeqIO

genome_file = "NC_000913.2.fasta"
genome_file_type = genome_file.split('.')[-1]
record = SeqIO.read(genome_file, genome_file_type)
length = len(record.seq)

## Подготовка данных

In [41]:
import numpy as np

def recode_data():
    bases_dict     = {"A": 0, "T": 1, "C": 2, "G": 3}
    bases_list = ["A", "T", "C", "G"]
    bases_np        = {
                        "A": np.array([1, 0, 0, 0], dtype = np.float32),
                        "T": np.array([0, 1, 0, 0], dtype = np.float32),
                        "C": np.array([0, 0, 1, 0], dtype = np.float32),
                        "G": np.array([0, 0, 0, 1], dtype = np.float32)
                        }

    seq_np = np.zeros((length, 4), dtype = np.float32) 
    for index in range(length):
        base = record.seq[index]
        if base in bases_list:
            channel = bases_dict[base]
            seq_np[index, channel] = 1
        else:
            print("alternative base")
    return seq_np

In [49]:
LENGTH = 10
N_TRAIN = 100000
N_TEST = 10000

def gen_data_sets(seq_np):
    
    train_ids = np.random.choice(int(length * (1-test)) - 300, N_TRAIN, replace=False) + 200
    test_ids = np.random.choice(int(length * test) - 300, N_TEST, replace=False) + int(length * (1-test))

    train1 = np.array([seq_np[x-LENGTH:x, ...] for x in train_ids])
    train2 = np.array([seq_np[x+1:x+LENGTH+1, ...] for x in train_ids])
    train_ans = np.array([seq_np[x, ...]  for x in train_ids])

    test1 = np.array([seq_np[x-LENGTH:x, ...] for x in test_ids])
    test2 = np.array([seq_np[x+1:x+LENGTH+1, ...] for x in test_ids])
    test_ans = np.array([seq_np[x, ...]  for x in test_ids])
    return [train1, train2, train_ans], [test1, test2, test_ans]

In [46]:
seq_np = recode_data()
[train1, train2, train_ans], [test1, test2, test_ans] = gen_data_sets(seq_np)
print(train1.shape, train2.shape, train_ans.shape)
print( test1.shape, test2.shape, test_ans.shape)

100000 222 4175601
10000 4175730 4639365
(100000, 10, 4) (100000, 10, 4) (100000, 4)
(10000, 10, 4) (10000, 10, 4) (10000, 4)


## Самая простая модель - скрытая марковская

https://hmmlearn.readthedocs.io/en/latest/tutorial.html